In [206]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv
from scipy.linalg import rq
import math

# INPUT DATA

In [207]:
# 3D points on checkerboard grid
X = np.array([
    [2,2,0],
    [3,5,0],
    [5,5,2],
    [5,2,3],
    [5,6,3],
    [5,4,1]
]
)


In [208]:
# 2D points on image taken
x = np.array([
    [356, 836],
    [581, 886],
    [682, 700],
    [528, 532],
    [789, 694],
    [580, 705]
]
)

# ESTIMATING AND DECOMPOSITION OF PROJECTION MATRIX

In [209]:
# Finds centroid of input and avg distance of points from centroid
def normalizing_params(pts):
    centroid=np.mean(pts,axis=0)
    avg_distance=np.mean(LA.norm(pts-centroid, axis=1))
    return centroid,avg_distance


In [210]:
# Obtaining the T,U normalizing matrices
def normalizing_matrix(pts):
    c,d=normalizing_params(pts)
    n=pts.shape[1]
    if n==2:
        d=d/math.sqrt(2)
        mat=np.array([[1/d,0,-c[0]/d],[0,1/d,-c[1]/d],[0,0,1]])
    else:
        d=d/math.sqrt(3)
        mat=np.array([[1/d,0,0,-c[0]/d],[0,1/d,0,-c[1]/d],[0,0,1/d,-c[2]/d],[0,0,0,1]])
    return mat

In [211]:
# Homogenizing the input coordinates
def homo_coord(pts):
    rows=pts.shape[0]
    pts=np.append(pts, np.ones((rows, 1)), axis=1)
    return pts

In [212]:
# Using fiducial points to obtain the equations needed to solve for P
def Fiducial_matrix(X_pts,x_pts):
    rows=X_pts.shape[0]
    X_pts_homo=homo_coord(X_pts)
    x_pts_homo=homo_coord(x_pts)

    T=normalizing_matrix(x_pts)
    U=normalizing_matrix(X_pts)
    x_pts_homo=(np.dot(T,x_pts_homo.T)).T
    X_pts_homo=(np.dot(U,X_pts_homo.T)).T

    
    zerocol=np.zeros((rows,4))
    multx=(-X_pts_homo.T*x_pts_homo[:,0]).T
    multy=(-X_pts_homo.T*x_pts_homo[:,1]).T

    P=np.append(X_pts_homo,zerocol,axis=1)
    P=np.append(P,multx,axis=1)
    P=np.append(P,zerocol,axis=1)
    P=np.append(P,X_pts_homo,axis=1)
    P=np.append(P,multy,axis=1)
    
    P=np.reshape(P,(2*rows,12))
    return P

In [213]:
#Denormalizing function for P using T and U
def denormalize(X_pts,x_pts,P):
    T=normalizing_matrix(x_pts)
    U=normalizing_matrix(X_pts)
    P=np.dot(inv(T),P)
    P=np.dot(P,U)
    return P

In [214]:
#Estimating P using svd decomposition and using vector corresponding to least eigenvalue
def P_estimate(X_pts,x_pts):
    P=Fiducial_matrix(X_pts,x_pts)
    u, s, vh = np.linalg.svd(P)
    least_val_vector=vh[-1]
    P=np.reshape(least_val_vector,(3,4))
    P=denormalize(X_pts,x_pts,P)
    return P

In [215]:
# RQ decomposition of P to get K,R,X_0
def P_decomposition(P):
    KR=P[:,0:3]
    K, R = rq(KR)
    K=K/K[2][2]
    KRX=P[:,-1]
    X_0 = np.dot(-np.linalg.inv(KR),KRX)
    
    return K,R,X_0

In [216]:
print(normalizing_matrix(x))#T

[[ 0.00898989  0.         -5.2680775 ]
 [ 0.          0.00898989 -6.52216762]
 [ 0.          0.          1.        ]]


In [217]:
print(normalizing_matrix(X))#U

[[ 0.79377167  0.          0.         -3.30738195]
 [ 0.          0.79377167  0.         -3.17508667]
 [ 0.          0.          0.79377167 -1.1906575 ]
 [ 0.          0.          0.          1.        ]]


In [218]:
P=P_estimate(X,x)
print(P)

[[ 2.54931107e+01  4.65679364e+01  1.05416476e+01  6.12003320e+01]
 [-9.99007970e+00  3.09516343e+01 -4.54633454e+01  4.40238385e+02]
 [ 3.11929016e-02  9.92867262e-03 -2.55718540e-02  4.94505055e-01]]


# SOME TESTING DATA

In [219]:
# 3D points for testing
X_test = np.array([
    [0, 0, 0],
    [3, 3, 0],
    [5, 4, 0],
    [5, 3, 2],
    [2, 6, 0],
    [4, 4, 0]
])

In [220]:
#2D points on image for testing
x_actual = np.array([
    [124, 887],
    [450, 815],
    [542, 750],
    [556, 624],
    [632, 980],
    [530, 798]
])

# USING PROJECTION MATRIX TO FIND PROJECTIONS OF ACTUAL 3D POINTS ON IMAGE

In [221]:
# return projected points
def test_pred(P, X_test):
  rows=X_test.shape[0]
  X_test = np.append(X_test, np.ones((rows,1)), axis=1)
  x_test = np.dot(P,X_test.T)
  x_test = x_test/x_test[2]
  x_test = x_test[0:2].T
  return x_test

In [222]:
# calculate the root mean square error
def rms_error(x_predict, x_actual):
    return np.sqrt(np.mean((x_predict-x_actual)**2))

In [223]:
#get all the intrinsic and extrinsic parameters from K,R,X_0
def params1(P):
    K,R,X0=P_decomposition(P)
    alpha=K[0][0]
    theta=np.arctan(-alpha/K[0][1])
    theta_degree=theta*180/np.pi
    beta=K[1][1]*np.sin(theta)
    x0=K[0][2]
    y0=K[1][2]
    skew=K[0][1]
    t=-np.dot(R,X0)
    return alpha,beta,theta_degree,x0,y0,skew,R,X0,K,t

In [224]:
alpha,beta,theta_degree,x0,y0,skew,R,X0,K,t=params1(P)

In [225]:
print(alpha,beta,theta_degree,x0,y0,skew,R,X0,K,t)

1170.2538403507795 -1166.1181878125326 88.92461280578931 572.5847972337257 671.2687401675427 -21.967129217040767 [[ 0.16899496  0.83160753  0.52902704]
 [ 0.63839356 -0.50129665  0.58408503]
 [ 0.750929    0.23902003 -0.61560951]] [-6.73385558 -0.13416143 11.07172423] [[ 1.17025384e+03 -2.19671292e+01  5.72584797e+02]
 [ 0.00000000e+00 -1.16632362e+03  6.71268740e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]] [-4.60768416 -2.23523305 11.90457339]


# INCORPORATING RADIAL DISTORTION

In [226]:
def test_pred2(P,X_test):
    rows=X_test.shape[0]
    X_test = np.append(X_test, np.ones((rows,1)), axis=1)
    x_test = np.dot(P,X_test.T)
    x_test = x_test/x_test[2]
    return x_test

In [227]:
arr=test_pred2(P,X)

In [228]:
#returns the radial distortion parameters
def get_param_val(P,rad_dist_params,K_inv):
    poly_val=np.zeros(rad_dist_params)
    distance_power_matrix=np.zeros((rad_dist_params,rad_dist_params))
    for i in range(rad_dist_params):
        x_one_pred=arr[:,i]
        x_one_actual=x.T[:,i]
        d2=LA.norm(np.dot(K_inv,x_one_pred))-1
        
        for j in range(rad_dist_params):
            distance_power_matrix[j][i]=np.power(d2,j+1)
        ratio_x_one=x_one_pred[:-1]/x_one_actual
        meanval=np.mean(ratio_x_one)
        polyval=meanval-1
        poly_val[i]=polyval
    dist_inv=inv(distance_power_matrix)
    param_val=np.dot(poly_val,dist_inv)
    return param_val

In [229]:
#predicting projections after considering distortion parameters
def rad_test_pred(P,X_test,rad_dist_params):

    # rad_dist_params=3   #<=6
    
    K,R,X0=P_decomposition(P)
    K_inv=inv(K)
    param_val=get_param_val(P,rad_dist_params,K_inv)
    # print(param_val)


    rows=X_test.shape[0]
    X_test = np.append(X_test, np.ones((rows,1)), axis=1)
    x_test = np.dot(P,X_test.T)
    x_test = x_test/x_test[2]

    K_inv=inv(K)
    cols=x_test.shape[1]
    for i in range(cols):
        d2=LA.norm(np.dot(K_inv,x_test[:,i]))-1
        mult=1
        lamb=1
        for j in range(rad_dist_params):
            mult*=d2
            lamb+=mult*param_val[j]
        x_test[0][i]/=lamb
        x_test[1][i]/=lamb
    return x_test[:2,:].T

# FINAL COMPUTED RESULTS

In [230]:
# for i in range(1,7):
#     print(i,rms_error(x,rad_test_pred(P,X,i)))

In [231]:
rms_error(x,test_pred(P,X)) #calibrate points without distortion

0.08674345300132424

In [232]:
rms_error(x,rad_test_pred(P,X,2)) #calibrate points with distortion

0.08673750582657837

In [233]:
rms_error(x_actual,test_pred(P,X_test)) #test points without distortion

2.372674321237422

In [234]:
rms_error(x_actual,rad_test_pred(P,X_test,2)) #test points with distortion

2.371977217568816

In [235]:
# for i in range(1,7):
#     print(i,rms_error(x_actual,rad_test_pred(P,X_test,i)))

# ALL POINTS

In [236]:
X_final = np.array([
    [0, 0, 0],
    [3, 3, 0],
    [5, 4, 0],
    [5, 3, 2],
    [2, 6, 0],
    [4, 4, 0],
    [2, 2, 0],
    [3, 5, 0],
    [5, 5, 2],
    [5, 2, 3],
    [5, 6, 3],
    [5, 4, 1]
]
)

In [237]:
arryy = rad_test_pred(P,X_final, 2)

In [238]:
arryy - np.array([0, 1600])

array([[  123.76005828,  -709.74456479],
       [  448.93521665,  -785.71336563],
       [  543.24283014,  -855.13434547],
       [  555.47035616,  -976.55570889],
       [  635.22714678,  -617.02484065],
       [  530.27183432,  -804.71695012],
       [  356.00005845,  -764.00013725],
       [  580.99986757,  -713.99979805],
       [  681.98750203,  -900.2449291 ],
       [  527.99990292, -1067.99997447],
       [  789.00669399,  -905.87444397],
       [  580.00615417,  -894.88047099]])